# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [10]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [12]:
def who_to_mark(edges, vertexes, V1, marked_V1, marked_V2):
    V2 = vertexes - V1
    min_result = 0
    win = [0] * len(vertexes) # как изменится разрез, если мы перенесем эту вершину в другую часть
    for e in edges: 
        if (e[0] in V2 and e[1] in V1) or (e[0] in V1 and e[1] in V2):
            win[e[0] - 1] -= 1
            win[e[1] - 1] -= 1
        else:
            win[e[0] - 1] += 1
            win[e[1] - 1] += 1
    V1_candidate, V2_candidate = -1, -1
    for v1 in V1 - marked_V1:
        for v2 in V2 - marked_V2:
            temp_result = win[v1 - 1] + win[v2 - 1] + 2 * ((v1, v2) in edges or (v2, v1) in edges)
            if (V1_candidate == -1 and V2_candidate == -1) or temp_result < min_result:
                min_result = temp_result     
                V1_candidate, V2_candidate = v1, v2
    return V1_candidate, V2_candidate 

In [17]:
def variable_depth_local_search(graph, depth):
    starting_point = set(range(1, len(graph[0]) // 2 + 1))
    edges = graph[1]
    vertexes = graph[0]
    marked_V1, marked_V2 = set(), set()
    for i in range(depth):
        v1, v2 = who_to_mark(edges, vertexes, starting_point, marked_V1, marked_V2)
        starting_point.add(v2)
        starting_point.remove(v1)
        marked_V1.add(v1)
        marked_V2.add(v2)
    return starting_point

In [20]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    depth = [2, 1, 1, 1]
    for i in range(4):
        filename = filenames[i]
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance, depth[i]))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [21]:
run_all()

Solving instance add20.graph… done in 3.6 seconds with quality 11098
Solving instance cti.graph… done in 9.9e+01 seconds with quality 94056
Solving instance t60k.graph…

KeyboardInterrupt: 

## Выводы
Все очень плохо и медленно работает :'(